In [69]:
import sys
if '/usr/users/vogg/.conda/envs/mktrack/lib/python3.8/site-packages' not in sys.path:
    sys.path.insert(0, '/usr/users/vogg/.conda/envs/mktrack/lib/python3.8/site-packages')
    sys.path.insert(0, '/usr/users/vogg/monkey-tracking-in-the-wild/src')
    sys.path.insert(0, '/usr/users/vogg/monkey-tracking-in-the-wild/src/lib')

import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision
from torchmetrics.detection import MeanAveragePrecision

import datasets.dataset as datasets
from models.model import create_model, load_model
from models.decode import mot_decode, map2orig

from opts import opts

In [70]:
models = ['macaquepose', 'macaquecp', 'macaquecpw']
seeds = ['1','2','3']
valsets = ['MacaquePose', 'MacaqueCopyPaste', 'MacaqueCopyPasteWild']

In [75]:
df = pd.DataFrame(columns = ['model', 'valset', 'seed', 'AP', 'AP50', 'AP75'])


for mdl in models:
    for valset in valsets:
        for seed in seeds:
            epoch = 150
            print(mdl, valset, seed)

            model = create_model('hrnet_32', heads =  {'hm': 1, 'wh': 2, 'id': 128, 'reg': 2}, 
                                 head_conv = 256,num_gc_cls = None, 
                                 clsID4GC = None)
            
            if mdl == 'macaquepose':
                epoch *= 2

            #model = load_model(model, '../models/mcqcp/model_150.pth')
            model = load_model(model, '../exp/mot/'+mdl+'_seed'+seed+'/model_'+str(epoch)+'.pth')
            model = model.to(torch.device('cuda'))
            model.eval()
            print('ok')

            path = "/usr/users/agecker/datasets/"

            dataloader = datasets.LoadImagesAndBoxes(path, 
                                                     "/usr/users/vogg/monkey-tracking-in-the-wild/src/data/macaquepose.val",
                                                    valset = valset)

            mean_ap = MeanAveragePrecision()

            for i, (img_path, img, img0,labels) in enumerate(dataloader):
                if i == 500:
                    break


                im_blob = torch.from_numpy(img).cuda().unsqueeze(0)

                width = img0.shape[1]
                height = img0.shape[0]
                inp_height = im_blob.shape[2]
                inp_width = im_blob.shape[3]
                h_out = inp_height // 4
                w_out = inp_width // 4


                with torch.no_grad():
                    output = model(im_blob)[-1]
                    hm = output['hm'].sigmoid_()
                    wh = output['wh']
                    id_feature = output['id']
                    id_feature = F.normalize(id_feature, dim=1)
                    reg = output['reg']

                dets, inds, cls_inds_mask = mot_decode(heatmap=hm,
                                                       wh=wh,
                                                       reg=reg,
                                                       num_classes=1,
                                                       cat_spec_wh=False,
                                                       K=50)

                # translate and scale
                dets = map2orig(dets, h_out, w_out, height, width, 1)
                dets = dets[0]

                remain_inds = dets[:, 4] > 0.01
                dets = dets[remain_inds]
                #print("Total detections: ", len(dets), " - gt: ", len(labels))

                #ground truth
                boxes = torch.Tensor(labels[:,2:6] * np.array([width, height, width, height]))
                boxes = torchvision.ops.box_convert(boxes, "cxcywh", "xyxy")



                mean_ap.update(
                    [dict(boxes=torch.Tensor(dets[:,0:4]), scores=torch.Tensor(dets[:,4]), labels=torch.ones(len(dets[:,5])))],
                    [dict(boxes=boxes, labels=torch.ones(len(boxes)))],
                )

            scores = mean_ap.compute()
            print({'model': mdl, 'valset': valset, 'seed': seed, 
                      'AP': scores['map'], 'AP50': scores['map_50'], 'AP75': scores['map_75']})
            df = pd.concat([df, pd.DataFrame({'model': mdl, 'valset': valset, 'seed': seed, 
                      'AP': scores['map'].numpy(), 'AP50': scores['map_50'].numpy(), 'AP75': scores['map_75'].numpy()}, index = [0])])

                #img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
                #plt.imshow(img0)
                #plt.show()

macaquepose MacaquePose 1
=> loading pretrained model ../models/hrnetv2_w32_imagenet_pretrained.pth
loaded ../exp/mot/macaquepose_seed1/model_300.pth, epoch 300
ok
{'model': 'macaquepose', 'valset': 'MacaquePose', 'seed': '1', 'AP': tensor(0.7122), 'AP50': tensor(0.9272), 'AP75': tensor(0.8412)}
macaquepose MacaquePose 2
=> loading pretrained model ../models/hrnetv2_w32_imagenet_pretrained.pth
loaded ../exp/mot/macaquepose_seed2/model_300.pth, epoch 300
ok
{'model': 'macaquepose', 'valset': 'MacaquePose', 'seed': '2', 'AP': tensor(0.7042), 'AP50': tensor(0.9190), 'AP75': tensor(0.8247)}
macaquepose MacaquePose 3
=> loading pretrained model ../models/hrnetv2_w32_imagenet_pretrained.pth
loaded ../exp/mot/macaquepose_seed3/model_300.pth, epoch 300
ok
{'model': 'macaquepose', 'valset': 'MacaquePose', 'seed': '3', 'AP': tensor(0.7194), 'AP50': tensor(0.9221), 'AP75': tensor(0.8389)}
macaquepose MacaqueCopyPaste 1
=> loading pretrained model ../models/hrnetv2_w32_imagenet_pretrained.pth
load

In [72]:
df['AP'] = [x.numpy() for x in df['AP']]
df['AP50'] = [x.numpy() for x in df['AP50']]
df['AP75'] = [x.numpy() for x in df['AP75']]

AttributeError: 'float' object has no attribute 'numpy'

In [76]:
df

,model,valset,seed,AP,AP50,AP75
0,macaquepose,MacaquePose,1,0.712185,0.927207,0.841179
0,macaquepose,MacaquePose,2,0.704243,0.918983,0.824679
0,macaquepose,MacaquePose,3,0.719421,0.922112,0.838874
0,macaquepose,MacaqueCopyPaste,1,0.619552,0.855979,0.707526
0,macaquepose,MacaqueCopyPaste,2,0.616710,0.846807,0.718792
0,macaquepose,MacaqueCopyPaste,3,0.615841,0.845483,0.696541
0,macaquepose,MacaqueCopyPasteWild,1,0.713818,0.918193,0.827958
0,macaquepose,MacaqueCopyPasteWild,2,0.714352,0.917515,0.828093
0,macaquepose,MacaqueCopyPasteWild,3,0.735609,0.925954,0.841424
0,macaquecp,MacaquePose,1,0.727968,0.928118,0.844652


In [77]:
df.to_csv("/usr/users/vogg/monkey-tracking-in-the-wild/models150_detection.csv")

In [39]:
len(boxes)

2

In [41]:
torch.Tensor(dets[:,0:4])

tensor([[  20.4627,  272.1553,  458.3413,  672.3766],
        [ 588.6373,   68.4057, 1009.8489,  637.6304]])

{'map': tensor(0.6525),
 'map_50': tensor(1.),
 'map_75': tensor(0.5050),
 'map_small': tensor(-1.),
 'map_medium': tensor(-1.),
 'map_large': tensor(0.6525),
 'mar_1': tensor(0.4500),
 'mar_10': tensor(0.6500),
 'mar_100': tensor(0.6500),
 'mar_small': tensor(-1.),
 'mar_medium': tensor(-1.),
 'mar_large': tensor(0.6500),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}